# Tutorial: Building a Custom Environment in Aviary

## Prerequisites

To get started, you’ll need to install `aviary`, `ldp`, and `pydantic`, all available on PyPI. Note that `aviary` is listed on PyPI as `fhaviary`. Run the command below to install these packages:


In [ ]:
!pip install fhaviary ldp pydantic

You will need to set the `OPENAI_API_KEY` or the corresponding key for any other API you wish to access.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your_API_key"

## Background

[Aviary](https://github.com/Future-House/aviary) is our framework supporting diverse language environments, where actions are tools available to agents. [LDP](https://github.com/Future-House/ldp) is our framework for creating and training language agents.

Below, we briefly define some key classes and concepts from these libraries for context:

**From Aviary**
- `Message`: Used by language agents and environments for communication. Messages include attributes like `content` ot `role` (`system`, `user`, `assistant`, `tool` ), matching OpenAI's conventions.
- `Environment`: An environment is a stateful system or "world" where an agent operates by taking actions. In Aviary, these actions are called tools. The environment presents states that the agent observes (totally or partially), prompting it to use tools to affect outcomes. Each action taken yields a reward and leads to a new state.
- `Tool`: Defines an environmental tool that an agent can use to accomplish its task. Each environment contains its own set of tools. Most tools take arguments and tools can be called in parallel.
- `ToolRequestMessage`: This is a specialized subclasses of `Message` used for tool requests. Typically, a language agent sends a `ToolRequestMessage` to the environment to request the execution of a specific tool. The role of `ToolRequestMessage` is always `assistant`.

**From LDP**
- `Agent`: An entity that interacts with the environment, mapping observations to tool request actions.
- `Op`: Represents an operation within the agent. LDP includes various operations (Ops), such as API LLM calls, API embedding calls, or PyTorch module handling. These operations form the compute graph.
- `OpResult`: the output of an `Op`.


## Defining a Custom Environment

The example below walks through defining a custom language agent environment in Aviary. 
We define a simple environment where an agent takes actions to modify a counter.

In [ ]:
from pydantic import BaseModel

from aviary.core import Environment, Message, Tool, ToolRequestMessage


# State in this example is simply a counter
class CounterEnvState(BaseModel):
    count: int


class CounterEnv(Environment[CounterEnvState]):
    """A simple environment that allows an agent to modify a counter."""

    async def reset(self) -> tuple[list[Message], list[Tool]]:
        """Initialize the environment with a counter set to 0."""
        self.state = CounterEnvState(count=0)

        # Target count
        self.target = 10

        # Create tools allowing the agent to increment and decrement counter
        self.tools = [
            Tool.from_function(self.incr),
            Tool.from_function(self.decr),
        ]

        # Return an observation message with the counter and available tools
        return [Message(content=f"Count to 10. counter={self.state.count}")], self.tools

    async def step(
        self, action: ToolRequestMessage
    ) -> tuple[list[Message], float, bool, bool]:
        """Executes the tool call requested by the agent."""
        obs = await self.exec_tool_calls(action)

        reward = int(self.state.count == self.target)

        # Returns observations, reward, done, truncated
        return obs, reward, reward == 1, False

    def incr(self) -> str:
        """Increment the counter."""
        self.state.count += 1
        return f"counter={self.state.count}"

    def decr(self) -> str:
        """Decrement the counter."""
        self.state.count -= 1
        return f"counter={self.state.count}"

## Evaluating an Agent on the Environment

Following the definition of our custom environment, we can now evaluate a language agent
on the environment using Aviary's sister library LDP (https://github.com/Future-House/ldp).

In [ ]:
from pydantic import BaseModel, Field

from aviary.core import Message, Tool


class AgentState(BaseModel):
    """Simple bucket to store available tools and previous messages."""

    tools: list[Tool] = Field(default_factory=list)
    messages: list[Message] = Field(default_factory=list)

In [ ]:
from ldp.agent import Agent
from ldp.alg import RolloutManager
from ldp.graph import LLMCallOp

from aviary.core import ToolRequestMessage


class SimpleAgent(Agent):
    def __init__(self, **kwargs: dict) -> None:
        self._llm_call_op = LLMCallOp(**kwargs)

    async def init_state(self, tools: list[Tool]) -> AgentState:
        return AgentState(tools=tools)

    async def get_asv(
        self, agent_state: AgentState, obs: list[Message]
    ) -> tuple[ToolRequestMessage, AgentState, float]:
        """Take an action, observe new state, return value."""
        action: ToolRequestMessage = await self._llm_call_op(
            config={"name": "gpt-4o", "temperature": 0.1},
            msgs=agent_state.messages + obs,
            tools=agent_state.tools,
        )
        new_state: AgentState = AgentState(
            messages=agent_state.messages + obs + [action.value],
            tools=agent_state.tools,
        )
        # Return action, state, value
        return action, new_state, 0.0

### Create a simple agent and perform rollouts on the environment

In [ ]:
agent: SimpleAgent = SimpleAgent()

runner: RolloutManager = RolloutManager(agent=agent)

trajectories: list[tuple] = await runner.sample_trajectories(
    environment_factory=CounterEnv,
    batch_size=2,
)

# End
PS: See also a more advanced tutorial on creating a language agent in the [LDP repo](https://github.com/Future-House/ldp/blob/main/tutorials/creating_a_language_agent.ipynb).